# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import *
from IPython.display import clear_output
from pythonosc import dispatcher, osc_server
import joblib

# Capturing accelerometer data through OSC

In [ ]:
data = pd.DataFrame(columns=['x', 'y', 'z'])

# To keep track for data collection
index = 0

x = 0
y = 0
z = 0

# Creating a function that will handle and classify accelerometer data
def acceleration_vector(address, args):
    global data
    global index
    global x
    global y
    global z
    if address.find('accelerometer/x') != -1:
        x = args
    elif address.find('accelerometer/y') != -1:
        y = args
    elif address.find('accelerometer/z') != -1:
        z = args
        data = data.append({'x': x, 'y': y, 'z': z}, ignore_index=True)
        clear_output(wait=True)
        print(index)
        index = index + 1

# Attaching the function to the dispatcher
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/accelerometer/*", acceleration_vector)

# Starting the OSC server
server = osc_server.ThreadingOSCUDPServer(('193.157.241.238', 8000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
# Closing the server
server.server_close()

# Saving the data to a .csv-file
data.to_csv('./data/data.csv')

# Using the dataset to train a regressor

In [2]:
#### Read dataset
dataset = pd.read_csv('./data/data.csv')

# Importing the columns with accelerometer (gravity) data on the three exis
rawdata = dataset[['x', 'y', 'z']].to_numpy()
inputs = np.empty((0,3))
target = np.empty((inputs.shape[0],2))

# Iterating through the entries of the dataset and creating associated target values
# The index edges for the postures have been found manually by visually inspecting the waveforms
for i in range(0,rawdata.shape[0]):
    if (2 <= i <= 1025):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,1.]]), axis=0) #looking at phone
    elif (1052 <= i <= 1990):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[1.,0.]]), axis=0) #face level to the left
    elif (2007 <= i <= 3010):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,1.]]), axis=0) #face level upwards
    elif (3050 <= i <= 3937):
        inputs = np.append(inputs, rawdata[i,:].reshape(1,-1), axis=0)
        target = np.append(target, np.array([[0.,0.]]), axis=0) #face level downwards


# Creating train/test split
inputs_train, inputs_test, target_train, target_test = sklearn.model_selection.train_test_split(inputs, target, test_size=0.1)

# Training the model
mlp = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=(8,4), max_iter=20000, activation='logistic')
mlp.fit(inputs_train, target_train)
target_predict =  mlp.predict(inputs_test)

# Print the number of misclassified samples, accuracy and complete report (using scikit learn metric tools) 
print('r2 score on individual targets',sklearn.metrics.r2_score(target_test, target_predict, multioutput='raw_values'))

r2 score on individual targets [0.99645624 0.98604923]


# Saving the model to a file, for use in our 'amen_control.py' application

In [4]:
# Saving the classifier model to file
joblib_file = "./data/mlp_model.pkl"
joblib.dump(mlp, joblib_file)

['./data/mlp_model.pkl']